In [ ]:
from omegaconf import OmegaConf
import omegaconf
from neuroformer.datasets import *

# config_path = "configs/tconf.yaml"
config_path = "configs/naturalmovie/tconf.yaml"
config = OmegaConf.load(config_path)
df, video_stack, train_dataset, test_dataset = instantiate_dataset(config)

In [18]:
config_path = "configs/naturalmovie/tconf.yaml"
config = OmegaConf.load(config_path)

In [19]:
from neuroformer.datasets import instantiate_model
model = instantiate_model(config, train_dataset)

30 363 {'sparse_mask': 0.0, 'p_sparse': 0.25, 'sparse_topk_frame': None, 'sparse_topk_id': None, 'n_dt': 3, 'data_size': 38001887, 'class_weights': None, 'pretrain': 0, 'n_state_layers': 4, 'n_state_history_layers': 0, 'n_stimulus_layers': 12, 'self_att_layers': 4, 'n_layer': 10, 'n_head': 2, 'n_embd': 256, 'n_embd_frames': 64, 'contrastive': 1, 'clip_emb': 1024, 'clip_temp': 0.5, 'conv_layer': True, 'kernel_size': [5, 8, 8], 'temp_emb': 1.0, 'pos_emb': 0.0, 'id_drop': 0.35, 'im_drop': 0.35, 'dt': 0.05, 'block_size': 30, 'id_block_size': 30, 'prev_id_block_size': 30, 'frame_block_size': 448}
114688
256 3
None


In [22]:
from neuroformer.datasets import instantiate_trainer
instantiate_trainer(config, model, train_dataset)

GPT(
  (tok_emb): Embedding(363, 256)
  (temp_emb): LearntTemporalEmbedding(
    (temp_emb): Sequential(
      (0): Linear(in_features=1, out_features=128, bias=True)
      (1): GELU(approximate='none')
      (2): Linear(in_features=128, out_features=256, bias=True)
      (3): Dropout(p=0.2, inplace=False)
    )
  )
  (temp_emb_prev): LearntTemporalEmbedding(
    (temp_emb): Sequential(
      (0): Linear(in_features=1, out_features=128, bias=True)
      (1): GELU(approximate='none')
      (2): Linear(in_features=128, out_features=256, bias=True)
      (3): Dropout(p=0.2, inplace=False)
    )
  )
  (frame_3d_emb): PositionalEncoding3D()
  (id_drop): Dropout(p=0.35, inplace=False)
  (im_drop): Dropout(p=0.35, inplace=False)
  (drop): Dropout(p=0.2, inplace=False)
  (video_encoder): VideoEncoder(
    (conv_block): Sequential(
      (0): Conv3d(1, 256, kernel_size=(5, 8, 8), stride=(5, 8, 8))
      (1): Rearrange('b e t h w -> b t h w e')
      (2): LayerNorm((256,), eps=1e-05, elementwise

In [23]:
len(df['ID'].unique())

360

In [24]:
train_dataset.id_population_size

363

In [25]:
model_dir = "models/tensorboard/natmovie/1_kernel/w:0.1_wp:0.1"

model_path = glob.glob(os.path.join(model_dir, "*.pt"))[0]
mconf_path = glob.glob(os.path.join(model_dir, "*_mconf.pkl"))[0]
tconf_path = glob.glob(os.path.join(model_dir, "*_tconf.pkl"))[0]

# model.load_state_dict(torch.load(model_path, map_location='cpu'), strict=True)

In [26]:
n_embd_frames = 64
kernel_size = (5, 8, 8)
frame_block_size = ((20 // kernel_size[0] * 64 * 112) // (n_embd_frames))
print(frame_block_size)

448


In [27]:
with open(mconf_path, "rb") as f:
    mconf = pickle.load(f)
with open(tconf_path, "rb") as f:
    tconf = pickle.load(f)

In [28]:
# print all attribites and values of the model
for attr in dir(mconf):
    if not attr.startswith("__"):
        print(attr, getattr(mconf, attr))

attn_pdrop 0.2
block_size 478
class_weights None
clip_emb 1024
clip_temp 0.5
contrastive True
conv_layer True
data_size 38001887
dt 0.05
embd_pdrop 0.2
epoch 105
frame_block_size 448
frame_window 1
id_block_size 15
id_drop 0.35
id_vocab_size 363
im_drop 0.35
itos_dt {0: 0.0, 1: 0.05, 2: 0.1}
kernel_size (5, 8, 8)
n_dt 3
n_embd 128
n_embd_frames 64
n_head 2
n_layer 10
n_state_history_layers 0
n_state_layers 4
n_stimulus_layers 10
neurons [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133

In [29]:
# print all attribites and values of the model
for attr in dir(tconf):
    if not attr.startswith("__"):
        print(attr, getattr(tconf, attr))

batch_size 320
betas (0.9, 0.95)
block_size 478
ckpt_path models/tensorboard/natmovie/1_kernel/w:0.1_wp:0.1/6_Cont:True_window:0.1_f_window:1_df:0.05_blocksize:15_sparseFalse_conv_True_shuffle:True_batch:320_sparse_(None_None)_blocksz478_pos_emb:False_temp_emb:True_drop:0.35_dt:True_2.0_0.1_max0.05_(4, 0, 10)_2_128_nembframe64_(5, 8, 8).pt
clip_norm 1.0
dataset higher_order
decay_weights True
final_tokens 3831626250
grad_norm_clip 1.0
id_block_size 15
learning_rate 0.0003
lr_decay True
max_epochs 250
mode predict
no_pbar False
num_workers 4
patience 3
plot_raster False
score_metrics ['precision', 'recall', 'F1']
show_grads False
shuffle True
warmup_tokens 8000000.0
weight_decay 0.1


In [30]:
isinstance(config['model'], omegaconf.dictconfig.DictConfig) 

# for name, value in config['model'].items():
#     print(name, value)

kwarg = {"config": config['model']}
for k, v in kwarg.items():
        if isinstance(v, omegaconf.dictconfig.DictConfig):
            for name, value in v.items():
                print(name, value)

sparse_mask 0.0
p_sparse 0.25
sparse_topk_frame None
sparse_topk_id None
n_dt 3
data_size 38001887
class_weights None
pretrain 0
n_state_layers 4
n_state_history_layers 0
n_stimulus_layers 12
self_att_layers 4
n_layer 10
n_head 2
n_embd 256
n_embd_frames 64
contrastive 1
clip_emb 1024
clip_temp 0.5
conv_layer True
kernel_size [5, 8, 8]
temp_emb 1.0
pos_emb 0.0
id_drop 0.35
im_drop 0.35
dt 0.05
block_size 30
id_block_size 30
prev_id_block_size 30
frame_block_size 448


In [ ]:
config.keys()

In [ ]:
for key in config:
    cf = config[key]
    for a in dir(cf):
        if not a.startswith('__'):
            globals()[a] = getattr(cf, a)

for attribute in dir(train_dataset):
    if not attribute.startswith("__"):
        globals()[attribute] = getattr(train_dataset, attribute)

neurons = df['ID'].unique()

mc = config

In [ ]:
"""

RUN SIMULATION

"""


from utils import *
from IPython.utils import io
# top_p=0.25, top_p_t=0.9, temp=2.

# for idx in range(len(df_pred['ID'])):
#     df_pred['ID'][idx] = itos[df_pred['ID'][idx]]

# model_path = "/home/antonis/projects/slab/git/neuroformer/models/tensorboard/V1_AL_modalities/cont_weighted/25_Cont:True_0.50.1_sparseFalse_conv_True_shuffle:True_batch:224_sparse_(None_None)_pos_emb:False_temp_emb:True_drop:0.2_dt:True_2.0_0.5_max0.1_(4, 4, 6)_2_256_nembframe64.pt"
# model_path = "/Users/antonis/Downloads/[16, 17, 18, 19]_Cont_True_0.50.05_sparseFalse_conv_True_shuffle_True_batch_224_sparse_(None_None)_pos_emb_False_temp_emb_True_drop_0.2_dt_True_2.0_0.5_max0.05_(4, 0, 6)_2_256_nembframe64-2.pt"
# model_path = "/Users/antonis/projects/slab/neuroformer/neuroformer/models/tensorboard/V1_AL/sos_clip/25_Cont:True_0.50.1_sparseFalse_conv_True_shuffle:True_batch:96_sparse_(200_4)_pos_emb:False_temp_emb:True_drop:0.2_dt:True_2.0_0.5_max0.1_(2, 2, 2)_2_256_nembframe64_(20, 8, 8).pt"
model.load_state_dict(torch.load(model_path, map_location='cpu'))

train_trials = train_dataset.data['Trial'].unique()
test_trials = test_dataset.data['Trial'].unique()
neurons = sorted(list(set(df['ID'].unique())))

train_data = df[df['Trial'].isin(train_trials)]
test_data = df[df['Trial'].isin(test_trials)]
trials = test_data['Trial'].unique()[0:2]
# trials = train_data['Trial'].unique()
# trials = train_data['Trial'].unique()[::4]

# trials = [start + (i * 20) for i in range(3)]
# trials = df['Trial'].unique()[0::20] 
results_dict = dict()
# for n in range(2, 20):
df_pred = None
df_true = None
n_p = 0.3   # (n + 1) * 0.05
temp = 2
# stoi['SOS'] = 2000
for trial in trials:    # test_data['Trial'].unique():
    # with io.capture_output() as captured:
        print(f"Trial: {trial}")
        df_trial = df[df['Trial'] == trial]
        trial_dataset = SpikeTimeVidData2(df_trial, None, block_size, id_block_size, frame_block_size, prev_id_block_size, 
                                          window, dt, frame_memory, stoi, itos, neurons, stoi_dt, itos_dt, frame_feats, 
                                          pred=False, window_prev=window_prev, frame_window=frame_window)
        trial_loader = DataLoader(trial_dataset, shuffle=False, pin_memory=False)
        results_trial = predict_raster_recursive_time_auto(model, trial_loader, window, window_prev, stoi, itos_dt, itos=itos, sample=True, top_p=0.95, top_p_t=0.95, temp=1.0, temp_t=1.0, frame_end=0, get_dt=True, gpu=False, pred_dt=True)
        # results_trial = predict_raster_hungarian(model, loader, itos_dt, top_p=0.75, temp=1)
        # print(f"MAX ID ---- {sorted(results_trial['ID'].unique()[-10])}")
        df_trial_pred, df_trial_true = process_predictions(results_trial, stoi, itos, window)
        print(f"pred: {df_trial_pred.shape}, true: {df_trial_true.shape}" )
        if df_pred is None:
            df_pred = df_trial_pred
            df_true = df_trial_true
        else:
            df_pred = pd.concat([df_pred, df_trial_pred])
            df_true = pd.concat([df_true, df_trial_true])

# df_preds[n] = df_pred
# print(f"--- n: {n}, n_p: {n_p}, temp: {temp} ---")
scores = compute_scores(df[df['Trial'].isin(trials)], df_pred)
print(scores)
print(f"pred: {len(df_pred)}, true: {len(df_true)}" )
# results_dict[n] = (scores)


In [ ]:
trials

In [ ]:
test_data

In [ ]:
trials = test_data['Trial'].unique()[0:2]
trials

In [ ]:
df_1 = df[df['Trial'].isin(trials)]
df_2 = df[df['Trial'].isin(trials + 1)]
df_3 = df[df['Trial'].isin(trials + 2)]

df_list = [df_pred, df_1, df_2, df_3]

for df_ in df_list:
    window_pred = 0.5
    df_['Interval'] = make_intervals(df_, window_pred)

window_pred = window if window_pred is None else window_pred
intervals = np.array(sorted(set(df['Interval'].unique()) & set(df['Interval'].unique())))
labels = np.array([round(window_pred + window_pred*n, 2) for n in range(0, int(max(df_pred['Interval']) / window_pred))])
ids = sorted(set(df['ID'].unique()) & set(df['ID'].unique()))

# labels = sorted(set(df_pred_full['Interval'].unique()))
rates_pred = get_rates_trial(df_pred, labels)
rates_1 = get_rates_trial(df_1, labels)
rates_2 = get_rates_trial(df_2, labels)
rates_3 = get_rates_trial(df_3, labels)

top_corr_pred = calc_corr_psth(rates_pred, rates_1)
top_corr_real = calc_corr_psth(rates_1, rates_2)
top_corr_real_2 = calc_corr_psth(rates_1, rates_3)

In [ ]:
"""

Evaluate results

"""

df_pred_full = df_pred 

from visualize import *

# df_2['Trial'] -= 2
id_pred, id_true_1, id_true_2 = len(df_pred_full['ID'].unique()), len(df_1['ID'].unique()), len(df_2['ID'].unique())
print(f"id_pred: {id_pred}, id_true_1: {id_true_1}, id_true_2: {id_true_2}")

len_pred, len_true = len(df_pred_full), len(df_1)
print(f"len_pred: {len_pred}, len_true: {len_true}")

accuracy = get_accuracy(df_pred, df_2)

scores = compute_scores(df_1, df_2)
pred_scores = compute_scores(df_1, df_pred_full)
print(f"real: {scores}")
print(f"pred: {pred_scores}")

set_plot_white()
plt.figure(figsize=(10, 10), facecolor='white')
plt.title('PSTH Correlations (V1 + AL)', fontsize=25)
plt.ylabel('Count (n)', fontsize=25)
plt.xlabel('Pearson r', fontsize=25)
plt.hist(top_corr_real, label='real - real2', alpha=0.6)
# plt.hist(top_corr_real_2, label='real - real3', alpha=0.6)
plt.hist(top_corr_pred, label='real - simulated', alpha=0.6)
plt.legend(fontsize=20)

dir_name = os.path.dirname(model_path)
plt.savefig(os.path.join(dir_name, 'psth_corr.svg'))

plot_distribution(df_1, df_pred)

total_scores = dict()
total_scores['real'] = scores
total_scores['pred'] = pred_scores